# Scraping data

* ## Import relevant packages

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import json
import pandas as pd
import time
import requests
import concurrent.futures

* ## Functions for scraping Boligsiden

In [ ]:
#### Get URL for Boligsiden search for specified period in selected Kommune

def get_url_boligsiden(kommune, startdate, enddate, p):
    url = 'http://www.boligsiden.dk/salgspris/solgt/alle/{}'
    params = '?periode.from={}&periode.to={}&displaytab=mergedtab&sort' \
             '=salgsdato&salgstype=%5Bobject%20Object%5D&kommune={}'
    full_url = url + params
    return full_url.format(p, startdate, enddate, kommune)

#### Get number of pages for Boligsiden search

def get_max_pages_boligsiden(url):
    options = webdriver.chrome.options.Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome('/Users/frederikmadsen/PycharmProjects/Thesis/chromedriver', options=options)
    driver.get(url)
    page_text = driver.find_element_by_class_name("salesprice-result").text

    last_page_num = (page_text.split("af ")[1]).split("\n")[0]
    return last_page_num

#### Get all address links on search page

def get_all_urls_on_page_boligsiden(url):
    options = webdriver.chrome.options.Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome('/Users/frederikmadsen/PycharmProjects/Thesis/chromedriver', options=options)
    driver.get(url)

    all_https = []
    with_reentries_https = []

    for elem in driver.find_elements_by_tag_name('a'):
        all_https.append(elem.get_attribute("href"))

    #bolig-links wanted appear multiple times, so we take away all single time occuring links
    for i in range(len(all_https)):
        if all_https[i] in all_https[:i]:
            with_reentries_https.append(all_https[i])

    #Take away first two entries, which are not bolig links
    with_reentries_https = with_reentries_https[2:]

    reduced_list = list(set(with_reentries_https))

    #To make sure no other links are included
    boliger_https = []
    condition = 'https://www.boligsiden.dk/adresse/'
    for i in reduced_list:
        if condition in i:
            boliger_https.append(i)

    return boliger_https

#### Get list of all address URLs for search

def get_all_links_boligsiden(kommune, startdate, enddate):
    # Returns first https-page with given variables
    first_page = get_url_boligsiden(kommune, startdate, enddate, 1)

    # Getting number of total pages
    total_pages = get_max_pages_boligsiden(first_page)

    # Empty lists
    link_to_all_pages = []
    list_of_all_pages = []

    # Collects a list with all the pages that we want to collect
    for x in tqdm(range(int(total_pages))):
        all_pages = get_url_boligsiden(kommune, startdate, enddate, x + 1)
        link_to_all_pages.append(all_pages)

        page_list = get_all_urls_on_page_boligsiden(link_to_all_pages[x])
        list_of_all_pages.extend(page_list)

    # Returns list with all the wanted url's
    return (list_of_all_pages)

#### Scrape information for single address on address URL 

def get_simple_single_page_boligsiden(url):

    url = url
    html = urlopen(url)
    soup = BeautifulSoup(html.read(), 'html.parser')
    head = str(soup.find('head'))
    try:
        json_string = re.search(r'__bs_addresspresentation__ = ([^;]*)', head).group(1)
        data = json.loads(json_string)
        df1 = pd.json_normalize(data)
        df2 = pd.DataFrame()
    except:
        json_string = re.search(r'__bs_propertypresentation__ = ([^;]*)', head).group(1)
        data = json.loads(json_string)
        df2 = pd.json_normalize(data)
        df1 = pd.DataFrame()

    return df1, df2

#### Collect scraped information for all addresses in two dataframes

def get_data_boligsiden(links):
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()

    for x in tqdm(range(0, len(links))):
        try:
            df_pages1, df_pages2 = get_simple_single_page_boligsiden(links[x])
            df1 = pd.concat([df1, df_pages1])
            df2 = pd.concat([df2, df_pages2])
        except:
            pass


    return df1, df2


* ## Functions for scraping DinGeo.dk

In [ ]:
#### Get DinGeo-URLs for all addresses in Boligsiden dataframes 

def get_geolinks1(df):
    df["dingeo_link"] = ""

    for x in range(0, len(df)):
        if '-' in (df['address.street'][x]):
            df['address.street'][x] = df['address.street'].str.split('-').str[0][x] + '--' \
                                      + df['address.street'].str.split('-').str[1][x]

        if ',' in (df['address.street'][x]):
            add_part = str(df['address.postalId'][x]) + '-' + df['address.city'][x].replace(" ", "-") + '/' \
                       + df['address.street'].str.split(',').str[0][x].replace(" ","-") + '/' \
                       + df['address.street'].str.split(', ').str[1][x].replace(".", "").replace(" ", "-")
            url = 'https://www.dingeo.dk/adresse/' + add_part
        elif 'Adressen er ikke tilgængelig' in (df['address.street'][x]):
            url = 'Utilgængelig'
        else:
            add_part = str(df['address.postalId'][x]) + '-' + df['address.city'][x].replace(" ", "-") + '/' \
                       + df['address.street'].str.split(',').str[0][x].replace(" ","-")
            url = 'https://www.dingeo.dk/adresse/' + add_part

        if '-lejl-' in url:
            url = url.replace('-lejl-','-')

        df['dingeo_link'][x] = url

    return df

def get_geolinks2(df):
    df["dingeo_link"] = ""

    for x in range(0, len(df)):
        if '-' in (df['property.address'][x]):
            df['property.address'][x] = df['property.address'].str.split('-').str[0][x] + '--' \
                                        + df['property.address'].str.split('-').str[1][x]

        if ',' in (df['property.address'][x]):
            ad_part = str(df['property.postal'][x]) + '-' + df['property.city'][x].replace(" ", "-") + '/' \
                      + df['property.address'].str.split(',').str[0][x].replace(" ","-") + '/' \
                      + df['property.address'].str.split(', ').str[1][x].replace(".", "").replace(" ", "-")
            url = 'https://www.dingeo.dk/adresse/' + ad_part
        elif 'Adressen er ikke tilgængelig' in (df['property.address'][x]):
            url = 'Utilgængelig'
        else:
            ad_part = str(df['property.postal'][x]) + '-' + df['property.city'][x].replace(" ", "-") + '/' \
                      + df['property.address'].str.split(',').str[0][x].replace(" ","-")
            url = 'https://www.dingeo.dk/adresse/' + ad_part

        if '-lejl-' in url:
            url = url.replace('-lejl-','-')

        df['dingeo_link'][x] = url

    return df

#### Scrape information for each individual address on DinGeo.dk

def dingeo_page(url):
    url = url

    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Dictionary
    data = {}
    data['dingeo_link'] = url
    try:
        data['Radonrisiko'] = [soup.find_all("div", {"id": 'radon'})[0].find_all("strong")[0].get_text()]
    except:
        pass

    if 'ikke registreret trafikstøj' in soup.find_all("div", {"id": 'trafikstoej'})[0].get_text():
        data['Støjmåling'] = ['Ingen trafikstøj']
    elif 'mangler desværre at indsamle trafikstøj' in soup.find_all("div", {"id": 'trafikstoej'})[0].get_text():
        data['Støjmåling'] = ['Mangler']
    else:
        data['Støjmåling'] = [soup.find_all("div", {"id": 'trafikstoej'})[0].find_all("b")[1].get_text()]

    data['Oversvømmelsesrisiko_skybrud'] = [soup.find_all("div", {"id": 'skybrud'})[0].find_all("b")[0].get_text()]
    data['Meter_over_havet'] = [soup.find_all("div", {"id": 'stormflod'})[0].find_all("b")[0].get_text()]

    table_0 = pd.read_html(str(soup.find_all('table')))[0].iloc[:, 0:2]
    table_0 = table_0.set_axis(['Tekst', 'Værdi'], axis=1, inplace=False)

    table_1 = pd.read_html(str(soup.find_all('table')))[1].iloc[:, 0:2]
    table_1 = table_1.set_axis(['Tekst', 'Værdi'], axis=1, inplace=False)

    table_2 = pd.read_html(str(soup.find_all('table')))[2].iloc[:, 0:2]
    table_2 = table_2.set_axis(['Tekst', 'Værdi'], axis=1, inplace=False)

    table_3 = pd.read_html(str(soup.find_all('table')))[3:-2]
    table_3 = pd.concat(table_3).iloc[:, 0:2]
    table_3 = table_3.set_axis(['Tekst', 'Værdi'], axis=1, inplace=False)

    table = pd.concat([table_0, table_1, table_2, table_3])

    table = table.loc[table['Tekst'].isin(['Anvendelse', 'Opførselsesår', 'Ombygningsår', 'Fredning',
                                           'Køkkenforhold', 'Antal Etager', 'Antal toiletter', 'Antal badeværelser',
                                           'Antal værelser',
                                           'Ydervægsmateriale', 'Tagmateriale', 'Varmeinstallation',
                                           'Bygning, Samlet areal', 'Boligstørrelse', 'Kælder', 'Vægtet Areal'])]
    mydict = dict(zip(table.Tekst, list(table.Værdi)))
    data.update(mydict)

    try:
        if 'ikke finde energimærke' in soup.find_all("div", {"id": 'energimaerke'})[0].get_text():
            data['Energimærke'] = ['Mangler']
        else:
            data['Energimærke'] = [soup.find_all("div", {"id": 'energimaerke'})[0].find_all("p")[0].get_text()[-3:-2]]
        data['Indbrudsrisiko'] = [soup.find_all("div", {"id": 'indbrud'})[0].find_all("u")[0].get_text()]
    except:
        pass

    try:
        if 'ikke fredet' in str(soup.find_all("div", {"id": 'fbb'})[0].find_all("h2")[0]):
            data['Bevaringsværdig'] = [0]
        elif 'Bygningen er Bevaringsværdig' in str(soup.find_all("div", {"id": 'fbb'})[0].find_all("h2")[0]):
            data['Bevaringsværdig'] = re.findall(r'\d+', str(soup.find_all("div", {"id": 'fbb'})[0].find_all("p")[4]))
        elif 'Fejl ved opslag af' in str(soup.find_all("div", {"id": 'fbb'})[0].find_all("h2")[0]):
            data['Bevaringsværdig'] = 'Mangler' #Seems to be flaw on site, all get mangler
        else:
            data['Bevaringsværdig'] = 'Ukendt'
    except:
        pass

    try:
        data['Største_parti'] = re.findall(r'valg/(.*?)(?<!\\).png',
                                           str(soup.find_all("div", {"id": 'valgdata'})[0].find_all('h2')[0]))
        data['Valgdeltagelse'] = \
        re.findall("\d+.\d+", str(soup.find_all("div", {"id": 'valgdata'})[0].find_all('p')[1]))[1]
        data['Afstemningsområde'] = [soup.find_all("div", {"id": 'valgdata'})[0].find_all("strong")[0].get_text()]
    except:
        pass

    try:
        url_vurdering = url + '/vurdering'
        resp_vurdering = requests.get(url_vurdering)
        soup_vurdering = BeautifulSoup(resp_vurdering.text, 'html.parser')
        data['AVM_pris'] = \
        soup_vurdering.find_all("div", {"id": 'avmnumber'})[0].get_text() #made correction
    except:
        pass

        # Make dataframe
    df_page = pd.DataFrame(data)

    return df_page

#### Collect all scraped data from DinGeo for the addresses and ad to Boligsiden-dataframes

def for_threading(url):

    try:
        df_pages = dingeo_page(url)
        # df_geo = pd.concat([df_geo, df_pages])
        #   time.sleep(1)
        return df_pages
    except:
        pass

def add_dingeo(df):

    url_list = df['dingeo_link'].tolist()

    df_geo = pd.DataFrame()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = tqdm(executor.map(for_threading, url_list))

        for result in results:
            df_geo = pd.concat([df_geo, result])


    df_Boligsiden_Dingeo = pd.merge(df, df_geo, how='inner', on='dingeo_link', right_index=False).drop_duplicates()

    return df_Boligsiden_Dingeo



* ## Functions for scraping hvorlangterder.dk

In [ ]:
#### Scrape information for single address from hvorlangterder.dk

def get_hvorlangterder(address):
    try:
        url = 'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?poitypes' \
              '=daycare,doctor,hospital,junction,metro,school,stop,strain,supermarket,train,library,pharmacy,coast' \
              ',forest,lake,airport,sportshall,publicbath,soccerfield,roadtrain&fromaddress=' + address \
              + '&mot=foot&token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInByaXZzIjogInIxWjByMEYwazZCdFdxUWNPVXlrQi95N' \
                'lNVcEp2MlFiZ3lYZXRxNEhZNFhPLzNZclcwK0s5dz09In0.fP4JWis69HmaSg5jVHiK8nemiCu6VaMULSGGJyK4D4PkWq4iA1' \
                '+nSHWMaHxepKwJ83sEiy9nMNZhv7BcktRNrA'
        resp = requests.get(url)
        cont = resp.json()
        df = pd.DataFrame(cont).loc[['routedmeters']]
        df['Location'] = address

        return (df)
    except:
        pass


#### Scrape data from hvorlangterder.dk for all adresses and merge with data from Boligsiden and DinGeo.dk
    
def add_hvorlangterder(df):


    df_hvorlangt = pd.DataFrame()

    for i in tqdm(range(0,len(df))):
        try:
            data = get_hvorlangterder(str(df['Location'][i]))
            df_hvorlangt = pd.concat([df_hvorlangt, data])
        except Exception:
            pass
        time.sleep(0.2)


    merged = pd.merge(df, df_hvorlangt, how='inner', on='Location', right_index=False).drop_duplicates()
    return merged



* ## Collecting all data

In [ ]:
    # All code below is commented out as not to risk running the very time consuming scraping process again
    
    
    ############### Get Boligsiden-data ###############

    # links = get_all_links_boligsiden('København', '2016-01-01', '2020-12-31') # This was also done for Frederiksberg
    #
    # with open('links_boligsiden_K.txt', 'w') as file:
    #        file.write(str(links))
    #
    # with open("links_boligsiden_K.txt", "r") as file:
    #   links = eval(file.readline())
    #
    # df1, df2 = get_data_boligsiden(links[30000:])
    # df1.to_csv('boligsiden_1_K7.csv', index=False)
    # df2.to_csv('boligsiden_2_K7.csv', index=False)
    # # The above was done 7 times for Copenhagen and 1 time for Frederiksberg, as to keep data frames small


    # # This gives data for 41024, where the df2's are dropped since they have missing data.
    # # In most cases this is due to that the unit is still for sale.
    # # Other addresses were likewise dropped due to problem with Boligsiden page.


    ############### Get Dingeo-data ##################

    # df_Boligsiden1 = pd.read_csv("boligsiden_1_K7.csv")
    # df_Boligsiden_Geo1 = get_geolinks1(df_Boligsiden1) #[:]) #Here we choose how many to do at a time
    # df_Boligsiden_Dingeo1 = add_dingeo(df_Boligsiden_Geo1)
    # # pd.set_option('display.max_columns', None)
    # # print(df_Boligsiden_Dingeo1)
    # df_Boligsiden_Dingeo1.to_csv('boligsiden_dingeo_1_K7.csv', index=False)

    # # Data is lost for adresses where there is a problem with the address name or missing data on the DinGeo-page.
    # # This results in a data set of 40657 addresses

    ########## Get Hvorlangterder data ##############

    # geo_bolig1 = pd.read_csv("boligsiden_dingeo_1_K7.csv")
    # geo_bolig1['Location'] = geo_bolig1['address.street'].str.split(',').str[0] + ', ' \
    #                        + geo_bolig1['address.postalId'].astype(str)
    # df_Boligsiden_Dingeo_Hvorlangterder1 = add_hvorlangterder(geo_bolig1)
    # df_Boligsiden_Dingeo_Hvorlangterder1.to_csv('bdh_1_K7.csv', index=False)

    # # Again loss of addresses, where no information is available on the hvorlangerder.dk web page
    # # This finally gives 40606 adresses.

    ########### Creating final raw data frame ##########

    # bdh_1_F = pd.read_csv("bdh_1_F.csv")
    # bdh_1_K1 = pd.read_csv("bdh_1_K1.csv")
    # bdh_1_K2 = pd.read_csv("bdh_1_K2.csv")
    # bdh_1_K3 = pd.read_csv("bdh_1_K3.csv")
    # bdh_1_K4 = pd.read_csv("bdh_1_K4.csv")
    # bdh_1_K5 = pd.read_csv("bdh_1_K5.csv")
    # bdh_1_K6 = pd.read_csv("bdh_1_K6.csv")
    # bdh_1_K7 = pd.read_csv("bdh_1_K7.csv")

    #
    # raw_data_1 = pd.concat([bdh_1_F, bdh_1_K1, bdh_1_K2, bdh_1_K3, bdh_1_K4,
    #                         bdh_1_K5, bdh_1_K6, bdh_1_K7], sort=False)
    # raw_data_1.to_csv('raw_data_1.csv', index=False)